In [29]:
!pip install langchain-community langchain-chroma langchain-openai bs4 pypdf PyPDF2

In [30]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from PyPDF2 import PdfReader
import requests
from io import BytesIO

In [31]:
llm = ChatOpenAI(model="gpt-4o-mini", api_key="API_KEY")

## [MY CODE]
해당 논무의 PDF 정보를 가져오는 코드입니다.

In [32]:
# PDF 파일 로드 (URL에서 로드)
pdf_url = "https://arxiv.org/pdf/2005.11401"
response = requests.get(pdf_url)
if response.status_code != 200:
    raise ValueError("Failed to download PDF from URL.")

pdf_stream = BytesIO(response.content)
reader = PdfReader(pdf_stream)

## [MY CODE]
논문은 Introduce - 실험 - 결과 - references 순서로 이루어졌습니다. 이 reference 참조 항목은 핵심적인 내용이 아니라고 생각하고 오히려 이후 RAG를 통해 정보를 추출할때 겹쳐서 가져와 전체적인 요약에 혼란을 줄수 있는걸 발견했습니다. 따라서 해당 항목은 필터링 하고 제거했습니다.

In [33]:
# References 이후 내용 제거 함수
def filter_references(pages):
    """References 이후의 내용을 제거한 페이지 리스트 반환"""
    filtered_pages = []
    for page in pages:
        if "References" in page:
            break
        filtered_pages.append(page)
    return filtered_pages

# PDF 텍스트 추출 및 References 제거
pages = [page.extract_text() for page in reader.pages]
filtered_pages = filter_references(pages)

## [MY CODE]
벡터 스토어에 저장합니다.

In [34]:
# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
documents = text_splitter.create_documents(filtered_pages)

if not documents:
    raise ValueError("No text splits were generated. Check the content of the loaded documents.")

# 벡터 스토어 생성
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=OpenAIEmbeddings(api_key="API_KEY")
)

## [MY CODE]
 1. 주요 아이디어<br/>
 2. 실험 및 결과<br/>
 3. 결론 <br/>
 4. 향후 방향성 및 기대효과<br/>
 순서로 gpt에 요청을 할것이기에 해당 항목들을 RAG를 사용하여 조회합니다.

In [35]:
retriever = vectorstore.as_retriever()

# 사용자 입력 예시
user_input = """
다음과 같은 순서로 요약 해줘.
 1. 주요 아이디어
 2. 실험 및 결과
 3. 결론 
 4. 향후 방향성 및 기대효과
"""

# RAG를 활용하여 논문 내용 조회
retrieved_docs = retriever.invoke(user_input)
retrieved_text = " ".join([doc.page_content for doc in retrieved_docs])

print("Retrieved text:", retrieved_text)

Retrieved text: ﬁnd evidence for this hypothesis by feeding the BART-only baseline with the partial decoding "The
Sun. BART completes the generation "The SunAlso Rises" isanovel bythis author of"The Sun
Also Rises" indicating the title "The Sun Also Rises" is stored in BART’s parameters. Similarly,
BART will complete the partial decoding "The SunAlso Rises" isanovel bythis author of"A
with "The SunAlso Rises" isanovel bythis author of"AFarewell toArms" . This example shows
how parametric and non-parametric memories work together —the non-parametric component helps
to guide the generation, drawing out speciﬁc knowledge stored in the parametric memory.
4.4 Fact Veriﬁcation
Table 2 shows our results on FEVER. For 3-way classiﬁcation, RAG scores are within 4.3% of
state-of-the-art models, which are complex pipeline systems with domain-speciﬁc architectures and
substantial engineering, trained using intermediate retrieval supervision, which RAG does not require.
6 ﬁnd evidence for this hypo

## [MY CODE]
다음과 같은 프롬프트를 작성했습니다. 첫번째 요약 순서.<br/>
1). 요약 순서<br/>
1.1) 아이디어는 무엇인지<br/>
1.2) 그 아이디어를 증명하기 위한 실험은 무엇을 했는지.<br/>
1.3) 결과는 무엇인지<br/>
1.4) 향후 방향성 및 기대효과는 무엇인지.<br/>
위와 같은 템플릿을 기반으로 진행하였습니다.<br/>
<br/>
두번째 규칙<br/>
1). 각 항목별 3 ~ 5문장으로 가장 가독성이 좋은 문장으로 추출.<br/>
2). 항목별로 줄바꿈 실행. 가독성을 위해서<br/>
3). 위 양식을 지키지 않으면 패널티를 줘서 해당 양식을 따르도록 유도.<br/>
<br/>
전체적으로, 프롬프트를 짧고 간결하게 구성하였습니다. 오히려 길게 구성하면 모델별로 다르겠지만 chat gpt4o-mini는 더 안좋은 성능을 발휘하는걸 체험했습니다.<br/>

In [38]:
# 요약 템플릿 설정
summary_prompt = f"""
내용: {retrieved_text[:1500]}

다음과 같은 순서로 요약 해줘.
 1. 주요 아이디어
 - 핵심 아이디어를 특히 강조해줘.
 2. 실험 및 결과
 - 실험 및 결과는 중요한 부분이니 자세히 요약해줘.
 3. 결론
 4. 향후 방향성 및 기대효과

특히, 각 항목별 요약은 3-5 문장으로 요약해줘. 각 줄별로 줄바꿈 처리를 해줘
해당 양식을 지키지 않을시에는 큰 벌점이 주어질거야.
"""

# LLM을 사용하여 요약 생성
summary = llm(summary_prompt)
print("Generated Summary:")
print(summary.content)

Generated Summary:
1. 주요 아이디어  
- 이 연구는 BART 모델의 기억 체계, 즉 매개변수적(parametric) 및 비매개변수적(non-parametric) 기억이 어떻게 상호작용하여 정보를 생성하는지를 탐구한다.  
- BART가 불완전한 입력("The Sun")을 받고 이를 완성하는 방식("The Sun Also Rises")은 해당 제목이 모델의 매개변수에 저장되어 있음을 나타낸다.  
- 이러한 현상은 비매개변수적 구성 요소가 특정 지식을 이끌어내도록 도와주며, 매개변수적 메모리와의 협력이 이루어짐을 보여준다.  

2. 실험 및 결과  
- 연구에서는 FEVER 데이터셋을 기반으로 RAG 모델의 성능을 평가하였고, 3-way 분류에서 RAG의 점수가 최신 기술 모델들보다 4.3% 이내의 차이를 보였다.  
- 이들 최신 모델은 도메인 특화 아키텍처와 복잡한 파이프라인 시스템을 필요로 하며, 중간 검색 감독을 통해 훈련되는 반면, RAG는 이러한 감독을 요구하지 않는다.  
- 실험 결과, BART 모델이 어떻게 정보를 효과적으로 생성하고 완성하는지에 대한 실증적인 증거가 제시되었다.  

3. 결론  
- BART 모델은 매개변수적 및 비매개변수적 메모리를 통합하여, 정보 생성을 위한 단순하면서도 효과적인 방법을 제공한다.  
- 비매개변수적 메모리의 도움으로, 모델이 특정 지식에 기반하여 정확한 문장을 생성할 수 있음을 확인하였다.  
- 이 연구는 자연어 처리에서 매개변수적 기억과 비매개변수적 기억의 통합적 접근이 갖는 잠재력을 강조한다.  

4. 향후 방향성 및 기대효과  
- 향후 연구에서는 비매개변수적 메모리의 더욱 다양한 적용 방법을 탐구하고, 이를 통해 더 정교한 생성 모델을 개발할 수 있을 것으로 기대된다.  
- 또한, BART 모델의 성능을 개선하고, 비매개변수적 기억의 기여를 더욱 명확히 하기 위한 실험이 필요하다.  
- 이러한 접근은 자연어 처리 분야에서 모델의 일반화 능력을 향상시키는 데 기여할 수 있

## [MY CODE] GPT 평가 - 95점
평가 기준 및 점수 <br/>
정확성 (30점):<br/>
<br/>
논문의 주요 내용을 정확히 반영했는지 평가.<br/>
주요 아이디어와 RAG 모델의 성능, 매개변수적 및 비매개변수적 메모리의 역할 등을 정확히 설명했습니다.<br/>
그러나 FEVER 외 다른 데이터셋 실험 결과가 생략되어 약간의 정보 손실이 있습니다.<br/>
점수: 28/30<br/>
간결성 (25점):<br/>
<br/>
중요한 내용을 간결하고 효과적으로 전달했는지 평가.<br/>
불필요한 설명 없이 핵심 내용을 잘 요약했습니다.<br/>
점수: 25/25<br/>
구조화 (20점):<br/>
<br/>
논리적이고 체계적으로 구성되었는지 평가.<br/>
주요 아이디어, 실험 결과, 결론, 향후 방향성을 명확히 구분하여 논리적인 흐름을 유지했습니다.<br/>
점수: 20/20<br/>
포괄성 (15점):<br/>
<br/>
논문의 다양한 측면(방법론, 결과, 결론 등)을 충분히 다뤘는지 평가.<br/>
FEVER 데이터셋 외에도 TriviaQA, WebQuestions, Natural Questions 등에서의 성능 비교가 빠져 있어 포괄성이 다소 부족합니다.<br/>
점수: 12/15<br/>
언어의 명료성 (10점):<br/>
<br/>
독자가 쉽게 이해할 수 있도록 명확하게 작성되었는지 평가.<br/>
간결하고 명확한 언어로 작성되었습니다.<br/>
점수: 10/10<br/>
총점: 95/100<br/>
개선 제안<br/>
실험 및 결과 부분에서 다른 데이터셋(TriviaQA, WebQuestions, Natural Questions 등)의 결과를 간략히 언급하여 포괄성을 높이는 것이 좋습니다.<br/>
향후 방향성에서 RAG 모델의 실제 활용 사례(예: 실시간 정보 업데이트 가능성)를 더 구체적으로 언급하면 더욱 완성도 높은 요약이 될 것입니다.<br/>